#Data Assignment


## Summary

In this note, I will build a variant of regression models to predict salary on test data. There are 1 million observations in train and test data, respectively. Each of them has 7 features (attributes): **X = companyId**, **jobType**, **degree**, **major**, **industry**, **yearsExperience** and **milesFromMetropolis**. The first five attributes are categorical, whereas the last two are numeric. The target variable **y = salary** is numeric too. Therefore, this is a supervised learning, regression problem. 

**NOTE:** Through the script, I will designate data from **train_features_2013-03-07.csv**+**test_salary_2013-03-07.csv** as `train_data`, and data from **test_features_2013-03-07.csv** as `task_data`. We only use `train_data` to train regression models, and use features from `task_data` to generate the salary table for this assignment's task.

Before using the data to build predictive models, in addition to examine whether data is clean, we implemented `pandas` to preprocess data: (1) `Join train_feature + train_salary` to form `train_data`. For each observation, we need to macth **X** and **y** by **jobId**. (2) `Convert categorical variables to dummy variables`. Since there are categorical variables in the data (e.g. **jobType** = 'SENIOR', 'CTO', 'CFO' etc), we need to convert the catgeorical variables to the one-hot-encoding representation. Then the number of features becomes to 94. (3) `Concatenate train_data and task_data`. Before converting, we need to concatenate train and task dataFrame, in order to guarantee they have consistent feature columns. After concatenating, the number of observations in the table becomes 2 million and then we convert the entire data with OHE representation. (4) `Fetch the first million observation as train_data for models`. Since only the first million observations have target values **y**, we only use the first half data `train_data` to train models, and the second half, `task_data`, to generate `test_salaries.csv` for the assignment.

Next I implemented `scikit-learn` library to build three regression models: (1) ridge regression $-$ a linear model with $L_2$ regularization, and two nonlinear models $-$ (2) decision tree and (3) random forest. To compare model performance, we split `train_data` into training (80%) and test (20%) datasets. Use the training set to train models and the test dataset to compare the models. In the linear model, I implement 10-fold cross-validation to search for optimal hyperparameter of the $L_2$ regularization. For the decision tree and random forest, I performed grid search to avoid overfitting. For model comparison, I used **two metics** $-$ **MSE**, mean-sqaure-errors and, $R^2$, the portion of data variance explained by the models. A lower **MSE** or higher (adjusted) $R^2$ implies better regression model performance.

For most supervised machine learning problems, I used to start with linear models $-$ simple and have easy interpretation. Then take the linear models as baseline, and further examine if non-linear models significantly improve accuracy. All the models show lower **MSE** than that from baseline $-$ mean salary. This tells the ML models have better job to describe data variance.

By comparing $R^2$ and **MSE**, we found that the most important feature to determine salary is **jobType** and then **yearExpereince**, **milesFromMetropolis** and **industry**....., since the model without **jobType** has the most significant drop on $R^2$ as well as increase on **MSE**, compared to the model using all features. In ridge regression, we can also list the coefficients to rank the feature importance and identify positive/negative contribution.

The content is as follows:
     0. Preprocess Data
     1. Split training data to training/test datasets
     2. Build Regression Models
             2.0 Baseline
             2.1 Ridge regression
                * Feature importance interpretation for ridge regression
             2.2 Decision Tree
             2.3 Random Forest
                * Feature importance interpretation for random forest
             2.4 Model Comparison
     3. Generate Salary Profile

In [87]:
import numpy as np
import pandas as pd
from sklearn import linear_model
from sklearn import grid_search
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.cross_validation import cross_val_score, train_test_split

from sklearn import preprocessing

import time

# 0. Preprocess Data

In [154]:
train_features = pd.read_csv("train_features_2013-03-07.csv")
train_salary = pd.read_csv("train_salaries_2013-03-07.csv")
task_features = pd.read_csv("test_features_2013-03-07.csv")

## 0.1 DataFrame

`train_features` has dimension:

In [3]:
train_features.shape

(1000000, 8)

In [94]:
train_features.head()

,jobId,companyId,jobType,degree,major,industry,yearsExperience,milesFromMetropolis
0,JOB1362684407687,COMP37,CFO,MASTERS,MATH,HEALTH,10,83
1,JOB1362684407688,COMP19,CEO,HIGH_SCHOOL,NONE,WEB,3,73
2,JOB1362684407689,COMP52,VICE_PRESIDENT,DOCTORAL,PHYSICS,HEALTH,10,38
3,JOB1362684407690,COMP38,MANAGER,DOCTORAL,CHEMISTRY,AUTO,8,17
4,JOB1362684407691,COMP7,VICE_PRESIDENT,BACHELORS,PHYSICS,FINANCE,8,16


On the other hand, `test_features` has dimension:

In [155]:
task_features.shape

(1000000, 8)

In [156]:
task_features.head()

,jobId,companyId,jobType,degree,major,industry,yearsExperience,milesFromMetropolis
0,JOB1362685407687,COMP33,MANAGER,HIGH_SCHOOL,NONE,HEALTH,22,73
1,JOB1362685407688,COMP13,JUNIOR,NONE,NONE,AUTO,20,47
2,JOB1362685407689,COMP10,CTO,MASTERS,BIOLOGY,HEALTH,17,9
3,JOB1362685407690,COMP21,MANAGER,HIGH_SCHOOL,NONE,OIL,14,96
4,JOB1362685407691,COMP36,JUNIOR,DOCTORAL,BIOLOGY,OIL,10,44


`train_salary` is a dataFrame to have **salary** and **jobId**.

In [148]:
train_salary.head()

,jobId,salary
0,JOB1362684407687,130
1,JOB1362684407688,101
2,JOB1362684407689,137
3,JOB1362684407690,142
4,JOB1362684407691,163


## 0.2 Examine if data is clean

We go through all data points to see whether there exists `Nan` or missing data. The following results show the data is clean.

In [149]:
train_features.isnull().sum()

jobId                  0
companyId              0
jobType                0
degree                 0
major                  0
industry               0
yearsExperience        0
milesFromMetropolis    0
dtype: int64

In [150]:
train_salary.isnull().sum()

jobId     0
salary    0
dtype: int64

In [157]:
task_features.isnull().sum()

jobId                  0
companyId              0
jobType                0
degree                 0
major                  0
industry               0
yearsExperience        0
milesFromMetropolis    0
dtype: int64

## 0.3 Join train_features and salary

In [208]:
train_data = pd.merge(train_features, train_salary, on='jobId')

## 0.4 Concatenate train and task data

In [321]:
data = pd.concat([train_data, task_features])

In [318]:
len(data)

2000000

Now we have entire 2 million observations in the data pool. We can summarize the categorical variables as follows

In [214]:
print ('number of companyId:', len(set(data['companyId'].tolist())))
print ('number of jobType:', len(set(data['jobType'].tolist())))
print ('number of degree:', len(set(data['degree'].tolist())))
print ('number of major:', len(set(data['major'].tolist())))
print ('number of indutry:', len(set(data['industry'].tolist())))

number of companyId: 63
number of jobType: 8
number of degree: 5
number of major: 9
number of indutry: 7


## 0.5 Convert categorical variables to OHE repesentation

In [322]:
data = data[['companyId','jobType', 'degree', 'major', 'industry',
            'yearsExperience','milesFromMetropolis', 'salary']]
data= pd.get_dummies(data)

Now there are 95 columns (including **salary**), so the number of features is 94. It used dummy variable. For example, **jobType = CEO**, **jobType = CFO** converts to (**jobType_CEO = 1**, **jobType_CFO = 0**) and (**jobType_CEO = 0**, **jobType_CFO = 1**), respectively.

In [161]:
data.shape

(2000000, 95)

In [269]:
data.head()

,yearsExperience,milesFromMetropolis,salary,companyId_COMP0,companyId_COMP1,companyId_COMP10,companyId_COMP11,companyId_COMP12,companyId_COMP13,companyId_COMP14,...,major_MATH,major_NONE,major_PHYSICS,industry_AUTO,industry_EDUCATION,industry_FINANCE,industry_HEALTH,industry_OIL,industry_SERVICE,industry_WEB
0,10,83,130.0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
1,3,73,101.0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
2,10,38,137.0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
3,8,17,142.0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,8,16,163.0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0


## 0.6 Fetch the first million as train_data, and the second as task_data

In [323]:
train_data = data.iloc[:1000000,:]
task_data = data.iloc[1000000:,:]

Now we designate the first 1 million data for `train_data` to train ML models. The second million `task_data` is only used to generate the salary table. In `task_data` the **salary** column is useless.

In [324]:
train_data.shape, task_data.shape

((1000000, 95), (1000000, 95))

# 1. Split training data to training/test datasets

In [108]:
features = list(train_data.columns)
features.remove('salary')

X_train, X_test, y_train, y_test = train_test_split(train_data[features], train_data['salary'], test_size=0.2);
X_train.shape, y_train.shape;
y_train = y_train.tolist();
y_test = y_test.tolist();

# 2. Regression Models

In this section, we built a baseline model: the mean salary and three regression models: (1) Ridge regression: linear regression model with $L_2$ regularization (2) Decision tree, and (3) Random Forest. All the ML models show lower **MSE** than the baseline; thus we can judge the ML models work. The best performance is given by random forest regressor, but it is time-consuming to train the model, and only slightly better than ridge regression. Therefore, if considering production and efficiency, I would suggest to implement the ridge regression as ML pipeline.


## 2.0 Baseline

Let's start with an easy baseline model. The predicted salary for each **jobId** is simply given by mean value of salaries in the test dataset

In [310]:
mean_salary = np.mean(y_test)
print ('MSE of baseline:', round(np.mean([(y_test[i]-mean_salary)**2 for i in range(len(y_test))]),2))

MSE of baseline: 1497.84


As long as predictive models we build give lower **MSE** than 1497.84, we can judge our models work since it is able to describes data variance.

## 2.1 Ridge Regressor

Before implementing the ridge regression model, we should standardize the features:

In [165]:
X_train_scaled = preprocessing.scale(X_train)
scaler = preprocessing.StandardScaler().fit(X_train)
X_test_scaled = scaler.transform(X_test)

In training models, I used 10-fold cross-validation for grid search on optimal hyperparameter of $L_2$ regularization, and chosen the one giving highest $R^2$ on the cross validation dataset as the optimal model.

In [194]:
t0 = time.time()

best_ridge = None
max_rscore = -1
best_lambda = -1
for lambda_i in range(20, 70, 2):
    ridge = linear_model.Ridge(alpha=lambda_i)
    cv_score = cross_val_score(ridge, X_train_scaled, y_train, cv=10)
    if np.mean(cv_score) > max_rscore:
        max_rscore = np.mean(cv_score)
        best_ridge = ridge
        best_lambda = lambda_i

best_ridge.fit(X_train_scaled, y_train)
print ('time spent for training:', time.time()-t0 )

time spent for training: 734.8082828521729


In [275]:
best_lambda

50

The optimal hyperparameter for regularization $\lambda =50$. Then we input test dataset on the model to find out $R^2=0.742$, meaning the model almost explained 75% data variance:

In [274]:
print ('model R square for RR:', best_ridge.score(X_test_scaled, y_test))

model R square for RR: 0.742484944309


Next we check the **MSE** on training/test datasets. We can see the **MSE** shows much lower than that from baseline:

In [201]:
train_pred = best_ridge_model.predict(X_train_scaled)
test_pred = best_ridge_model.predict(X_test_scaled)
print ('MSE for training set:', round(np.mean([(train_pred[i]-y_train[i])**2 for i in range(len(train_pred))]),2),
      ', MSE for test set:', round(np.mean([(test_pred[i]-y_test[i])**2 for i in range(len(test_pred))]),2))

MSE for training set: 384.15 , MSE for test set: 385.67


### Feature importance interpretation for ridge regression

Next we will interpret the feature importance. In the linear regression, with standardized data, we can directly compare the predictor coefficients as importance. We show the coefficient magnitudes in the descending order:

In [325]:
weights = best_ridge.coef_
v = sorted(range(len(weights)), key=lambda k: weights[k], reverse=True)
sorted_features = [features[i] for i in v]
sorted_weights = [weights[i] for i in v]
df = pd.DataFrame({'feature': sorted_features, 'weight': sorted_weights})
df.head(5)

,feature,weight
0,yearsExperience,14.483164
1,jobType_CEO,9.160693
2,jobType_CFO,5.928725
3,jobType_CTO,5.906185
4,industry_OIL,5.158926


In [279]:
df.tail(4)

,feature,weight
90,industry_EDUCATION,-5.773449
91,jobType_JUNIOR,-7.303432
92,jobType_JANITOR,-11.469931
93,milesFromMetropolis,-11.536242


showing **yearExpereince** plays the most important positive role for a single feature in the regression model $-$ more **yearExpereince** contributes higher salaries. On the other hand, **milesFromMetropolis** has the most negative contribution to regression $-$ higher **milesFromMetropolis** corresponds to lower salaries. 

However, we have to notice that the majority of features came from OHE representation, like both **jobType_CEO** and **jobType_CFO** from **jobType**, so the importance of the features has been 'diluted'. To further explore the feature importance, we simply build a variant simple regression models which considers all features except the ones we are interested in. If (adjusted) $R^2$ reduces a lot or **MSE** increases a lot, it means the absent feature is important to explain model variance.

For exmaple, the $R^2$, adjusted $R^2$ and **MSE** without **yearExpereince** are

In [303]:
features_2 = list(X_train.columns)
features_2.remove('yearsExperience')
reg = linear_model.LinearRegression().fit(X_train[features_2], y_train)
R_sq = reg.score(X_test[features_2], y_test)
print ('R_sq =', round(R_sq,3), ', adjusted R_sq=', round(1-(1-R_sq)*(1000000)/(1000000-93-1),3))
pred = reg.predict(X_test[features_2])
print ('MSE:', round(np.mean([(pred[i]-y_test[i])**2 for i in range(len(pred))]),2))

R_sq = 0.601 , adjusted R_sq= 0.601
MSE: 597.49


On the other hand, without **milesFromMetropolis**, $R^2$, adjusted $R^2$ and **MSE** are

In [306]:
features_2 = list(X_train.columns)
features_2.remove('milesFromMetropolis')
reg = linear_model.LinearRegression().fit(X_train[features_2], y_train)
R_sq = reg.score(X_test[features_2], y_test)
print ('R_sq =', round(R_sq,3), ', adjusted R_sq=', round(1-(1-R_sq)*(1000000-1)/(1000000-93-1),3))
pred = reg.predict(X_test[features_2])
print ('MSE:', round(np.mean([(pred[i]-y_test[i])**2 for i in range(len(pred))]),2))

R_sq = 0.654 , adjusted R_sq= 0.654
MSE: 518.83


Next we look up the importance of the categorical features: **jobType**, **industry**, **degree**, **major** and **companyId**. For convenience, we define the following function to build regression models without the above features, and compute $R^2$, adjusted $R^2$ and **MSE** for different models:

In [307]:
def compute_R_sq(feature):
    features_2 = list(X_train.columns)
    types = [x for x in features_2 if x.split("_")[0] == feature]
    features_2 = [x for x in features_2 if x not in types]
    reg = linear_model.LinearRegression().fit(X_train[features_2], y_train)
    R_sq = reg.score(X_test[features_2], y_test)
    #print (feature, 'R_sq =', round(R_sq,3), ', adjusted R_sq=', round(1-(1-R_sq)*(len(y_test)-1)/(len(y_test)-(94-len(types))-1),3))
    pred = reg.predict(X_test[features_2])
    print (feature, 'R_sq =', round(R_sq,3), ', adjusted R_sq=', round(1-(1-R_sq)*(len(y_test)-1)/(len(y_test)-(94-len(types))-1),3), 
           ', MSE:', round(np.mean([(pred[i]-y_test[i])**2 for i in range(len(pred))]),2))

$R^2$, adjusted $R^2$ and **MSE** for **jobType**, **industry**, **degree**, **major** and **companyId** are respectively

In [308]:
compute_R_sq("jobType")
compute_R_sq("industry")
compute_R_sq("degree")
compute_R_sq("major")
compute_R_sq("companyId")

jobType R_sq = 0.486 , adjusted R_sq= 0.486 , MSE: 769.61
industry R_sq = 0.655 , adjusted R_sq= 0.655 , MSE: 516.71
degree R_sq = 0.73 , adjusted R_sq= 0.73 , MSE: 403.72
major R_sq = 0.735 , adjusted R_sq= 0.735 , MSE: 397.48
companyId R_sq = 0.743 , adjusted R_sq= 0.742 , MSE: 385.69


The model without **jobType** shows the most significant drop on $R^2$ as well as most significant increase on MSE (compared to $R^2=0.7424$ and MSE = 385.67 using all features). Hence we can conclude that the most important feature to determine the salary is **jobType**, second are **yearExpereince**, and then **milesFromMetropolis**,  **industry**... The least feature is **companyId**. 

The following is the comparison of models without the features:

| w/o feature | MSE increase | $R^2$ drop |
|-------|-----| ------|  
| **jobType**   |383.94|  0.256  |    
| **yearsExperience**   |211.82|  0.141  |     
| **milesFromMetropolis**   |133.16| 0.088  |     
|**industry** | 131.04 | 0.087 |  
| **degree**  | 18.05 | 0.012 |  
| **major**    | 11.81  | 0.007 |  
| **companyId** | 0.02  | ~ 0  | 

## 2.2 Decision Tree Regressor

For the decision tree regressor, we do grid search for maximum depth and minimum number of samples on leaf. Due to limited time, here I only use 5-fold cross-validation.

In [253]:
t0 = time.time()
decTreeReg = DecisionTreeRegressor(random_state=10)
parameters = {'max_depth':[5,15,20,25,30],'min_samples_leaf':[5,10,20,30]}
best_tree = grid_search.GridSearchCV(decTreeReg, parameters, cv=5, verbose=0, n_jobs=-1);
best_tree.fit(X_train, y_train);
print ('time spent for training:', time.time()-t0)

time spent for training: 954.8653700351715


In [255]:
best_tree.best_estimator_

DecisionTreeRegressor(criterion='mse', max_depth=20, max_features=None,
           max_leaf_nodes=None, min_impurity_split=1e-07,
           min_samples_leaf=30, min_samples_split=2,
           min_weight_fraction_leaf=0.0, presort=False, random_state=10,
           splitter='best')

So the best decision tree is given by max_depth=20 and min_samples_leaf=30. Decision tree regressor gives similar $R^2$ but slightly lower **MSE** on test dataset.

In [273]:
print ('model R square for DT:', best_tree.score(X_test, y_test))

model R square for DT: 0.742831298822


In [263]:
train_pred = best_tree.predict(X_train)
test_pred = best_tree.predict(X_test)
print ('MSE for training set:', round(np.mean([(train_pred[i]-y_train[i])**2 for i in range(len(train_pred))]),3),
      ', MSE for test set:', round(np.mean([(test_pred[i]-y_test[i])**2 for i in range(len(test_pred))]),3))

MSE for training set: 347.643 , MSE for test set: 385.199


## 2.3 Random Forest Regressor

For the random forest regressor, it is easy to get overfitting results. Therefore, grid search is important. Due to limited time, here I do one-time training (wihtout cross-validation). In addition to maximum depth and minimum number of sample, we also grid search on number of predictors (estimators).

In [312]:
t0 = time.time()
rfReg = RandomForestRegressor()
parameters = {'n_estimators': [25, 50, 75, 94], 'max_depth':[5,10,15], 'min_samples_leaf':[5,10]}
best_rfReg = grid_search.GridSearchCV(rfReg, parameters, verbose=0, n_jobs=-1)
best_rfReg.fit(X_train, y_train)
print ('time spent for training:', time.time()-t0)

time spent for training: 11650.994484901428


In [313]:
best_rfReg.best_estimator_

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=15,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=10,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=94, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [314]:
print ('mode R square for RF:', best_rfReg.score(X_test, y_test))

mode R square for RF: 0.743769542091


In [315]:
train_pred = best_rfReg.predict(X_train)
test_pred = best_rfReg.predict(X_test)
print ('MSE for training set:', round(np.mean([(train_pred[i]-y_train[i])**2 for i in range(len(train_pred))]),2),
      ', MSE for test set:', round(np.mean([(test_pred[i]-y_test[i])**2 for i in range(len(test_pred))]),2))

MSE for training set: 362.68 , MSE for test set: 383.79


### Feature importance interpretation for random forest

We can use random forest to determine feature importance. The importance unforunately cannot tell positive or negative contribution. For example, the largest magnitude is **jobType_JANITOR**, which corresponds to second negative importance, and on the other hand, second rank **yearsExperience** has the most positive importance for a single feature.

In [292]:
importance = best_rfReg.best_estimator_.feature_importances_
features = list(X_train.columns)

v = sorted(range(len(importance)), key=lambda k: importance[k], reverse=True)
sorted_importance = [importance[i] for i in v]
sorted_features = [features[i] for i in v]

df_importance = pd.DataFrame({'variable': sorted_features, 'importance' : sorted_importance})
df_importance.sort_index().head(5)

,importance,variable
0,0.259482,jobType_JANITOR
1,0.188460,yearsExperience
2,0.133651,milesFromMetropolis
3,0.094746,jobType_JUNIOR
4,0.070744,major_NONE


## 2.4 Model Comparison

Here we summarize the regression performance on the test dataset:

| Model | MSE | $R^2$ | $T_{\textrm{train}}$ (min) |
|-------|-----| ------| ------------ |  
| Ridge regression   |385.67|    0.742 |     12        |
| Decision tree   |385.20|   0.743 |     16        |
| Random forest   |383.79|  0.744 |     194      |

Though the random forest regressor outperforms to the others, it is not necessary to be useful in production. The training time shows it is not efficient (more than 3 hours). The ridge regression model is good enough to capture the data variance and predict the salaries.

## 3. Generate Salary Profile

With the optimal model, we are able to generate the salaries for each **jobId** using features from `task_data`. The following results show first 10 salaries using three regression models; we can see they roughly give similar salaries (at least within reasonable range). We still choose random forest regressor `best_rfReg` to generate `test_salaries.csv`. 

In [285]:
best_ridge.predict(scaler.transform(task_data[features]))[:10]

array([ 115.86049039,   92.26284307,  166.84533489,  105.7062408 ,
        119.16529486,  158.40777886,   98.35967685,  118.84023827,
        104.37623512,  108.15641365])

In [283]:
best_tree.predict(task_data[features])[:10]

array([ 124.19444444,   90.44186047,  181.2195122 ,  105.79069767,
        111.38571429,  147.28      ,   96.48648649,  123.74358974,
         96.83333333,  100.47368421])

In [326]:
best_rfReg.predict(task_data[features])[:10]

array([ 114.34513683,   92.30406848,  167.84192583,  107.91269304,
        113.03449735,  151.27921345,   99.46773519,  123.75761675,
        105.01468328,  100.81248462])

In [327]:
task_prediction = best_rfReg.predict(task_data[features])
df = pd.DataFrame({"jobId": task_features['jobId'].tolist(), "salary": list(task_prediction)})

In [328]:
df.head()

,jobId,salary
0,JOB1362685407687,114.345137
1,JOB1362685407688,92.304068
2,JOB1362685407689,167.841926
3,JOB1362685407690,107.912693
4,JOB1362685407691,113.034497


In [329]:
df.to_csv("test_salaries.csv", header=True, index = False)